In [1]:
import datetime as dt
import itertools
from pathlib import Path

import cartopy.crs as ccrs
import dask
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
import matplotlib.animation
import numpy as np
import pandas as pd
from scipy import stats
from scipy import interpolate
import xarray as xr

from remake import load_remake

from mcs_prime import McsTracks, McsTrack, PATHS

# %matplotlib inline

In [2]:
rmk = load_remake('../../remakefiles/plotting.py')

In [3]:
rmk.PlotCombinedMcsLocalEnvPrecursorMeanValueFilteredDecomp.tasks

[322132c675 PlotCombinedMcsLocalEnvPrecursorMeanValueFilteredDecomp(year=2020, decomp_mode='all', radius=100),
 63215edd8a PlotCombinedMcsLocalEnvPrecursorMeanValueFilteredDecomp(year=2020, decomp_mode='all', radius=200),
 8a1fc39139 PlotCombinedMcsLocalEnvPrecursorMeanValueFilteredDecomp(year=2020, decomp_mode='all', radius=500),
 3e8356b2a5 PlotCombinedMcsLocalEnvPrecursorMeanValueFilteredDecomp(year=2020, decomp_mode='all', radius=1000)]

In [5]:
t = rmk.PlotCombinedMcsLocalEnvPrecursorMeanValueFilteredRadius.tasks[0]

In [6]:
t

2a5019838b PlotCombinedMcsLocalEnvPrecursorMeanValueFilteredRadius(years=[2001, 2002, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [11]:
paths = [path for key, path in t.inputs.items() if 'mcs_local_env_2020' in key]

In [12]:
paths

[PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2020/01/lifecycle_mcs_local_env_2020_01.nc'),
 PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2020/02/lifecycle_mcs_local_env_2020_02.nc'),
 PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2020/03/lifecycle_mcs_local_env_2020_03.nc'),
 PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2020/04/lifecycle_mcs_local_env_2020_04.nc'),
 PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2020/05/lifecycle_mcs_local_env_2020_05.nc'),
 PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2020/06/lifecycle_mcs_local_env_2020_06.nc'),
 PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2020/07/lifecycle_mcs_local_env_2020_07.nc'),
 PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/mcs_local_envs/2020/08/lifecycle_mcs_local_env_2020_0

In [13]:
ds_full = xr.open_mfdataset(paths, combine='nested', concat_dim='tracks')

In [14]:
ds_full['tracks'] = np.arange(0, ds_full.dims['tracks'], 1, dtype=int)

In [15]:
ds_full.sel(radius=1000)

<xarray.Dataset>
Dimensions:                                             (tracks: 32138,
                                                         times: 424,
                                                         percentile: 5)
Coordinates:
  * tracks                                              (tracks) int64 0 ... ...
    radius                                              int64 1000
  * times                                               (times) int64 -24 ......
  * percentile                                          (percentile) int64 10...
Data variables: (12/26)
    mean_cape                                           (tracks, times) float64 dask.array<chunksize=(2406, 424), meta=np.ndarray>
    percentile_cape                                     (tracks, percentile, times) float64 dask.array<chunksize=(2406, 5, 424), meta=np.ndarray>
    mean_tcwv                                           (tracks, times) float64 dask.array<chunksize=(2406, 424), meta=np.ndarray>
    percentile_tcwv                                     (tracks, percentile, times) float64 dask.array<chunksize=(2406, 5, 424), meta=np.ndarray>
    mean_cin                                            (tracks, times) float64 dask.array<chunksize=(2406, 424), meta=np.ndarray>
    percentile_cin                                      (tracks, percentile, times) float64 dask.array<chunksize=(2406, 5, 424), meta=np.ndarray>
    ...                                                  ...
    mean_theta_e_mid                                    (tracks, times) float64 dask.array<chunksize=(2406, 424), meta=np.ndarray>
    percentile_theta_e_mid                              (tracks, percentile, times) float64 dask.array<chunksize=(2406, 5, 424), meta=np.ndarray>
    mean_delta_3h_cape                                  (tracks, times) float64 dask.array<chunksize=(2406, 424), meta=np.ndarray>
    percentile_delta_3h_cape                            (tracks, percentile, times) float64 dask.array<chunksize=(2406, 5, 424), meta=np.ndarray>
    mean_delta_3h_tcwv                                  (tracks, times) float64 dask.array<chunksize=(2406, 424), meta=np.ndarray>
    percentile_delta_3h_tcwv                            (tracks, percentile, times) float64 dask.array<chunksize=(2406, 5, 424), meta=np.ndarray>
Attributes:
    created by:           /home/users/mmuetz/projects/mcs_prime/ctrl/remakefi...
    calling file source:  """Remakefile to calculate local environments based...
    project repository:   https://github.com/markmuetz/MCS_PRIME
    remake version:       ['0', '6', '3', '0', 'beta']
    remake repository:    https://github.com/markmuetz/remake
    task:                 cb87fdf11a LifecycleMcsLocalEnv(year=2020, month=1)
    task doc:             Capture the env at various radii over the lifecycle...
    created on:           2023-11-16 21:14:59.739292
    nodename:             host638.jc.rl.ac.uk
    hostname:             jasmin
    output path:          /gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_outpu...
    contact:              mark.muetzelfeldt@reading.ac.uk

In [16]:
tracks = McsTracks.mfopen(t.inputs['tracks_2020'], None)

In [17]:
natural = tracks.dstracks.start_split_cloudnumber.values == -9999

In [18]:
natural.sum()

20595

In [19]:
mean_lat = np.nanmean(tracks.dstracks.meanlat.values, axis=1)

In [20]:
eqtrop = (mean_lat <= 30) & (mean_lat >= -30)

In [22]:
mean_landfrac = np.nanmean(tracks.dstracks.pf_landfrac.values, axis=1)
land = mean_landfrac > 0.5

In [26]:
ds_1000_nat_eqtrop_land = ds_full.sel(radius=1000).isel(tracks=(natural & eqtrop & land))

In [27]:
cin = ds_1000_nat_eqtrop_land.mean_cin.values

In [28]:
cin

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [31]:
(~np.isnan(cin)).sum()

15